**{DEV}\[TODO\]:**
- Check devnotes
- Proofread

---
### Chapter 1: System Modelling
---
# **Tutorial 1.b: Lagrange Mechanics with Non-Conservative Forces**
**Aim:** To incorporate non-conservative forces into a dynamic model.

Previously, we simulated a double pendulum swinging freely. Now we're going to add a force acting on the end in an arbitrary direction, and a control torque acting at joint 2:
<img src = "pendulum_force2.png" width = "250">

### **Contents**:
* [The Jacobian](#The-Jacobian)
* [Generalized Forces](#Generalized-Forces)
    * [Example: Forces](#Example:-Forces)
    * [Example: Torques](#Example:-Torques)

In [ ]:
# CODE FROM LAST TIME

# import libraries
import sympy as sym
import numpy as np

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>")) #ignore this, it'll be useful in tut 3

#--------------------------------------------------------------------------------------------------

# create symbolic variables

# system parameters
g = sym.symbols('g')
m1 ,  m2 = sym.symbols([ 'm_{1}', 'm_{2}']) # mass of links
l1 ,  l2 = sym.symbols([ 'l_{1}', 'l_{2}']) # length of links
d1 ,  d2 = sym.symbols([ 'd_{1}', 'd_{2}']) # distance to COM (as a fraction)
In1, In2 = sym.symbols(['In_{1}','In_{2}']) # moment of intertia of links

# generalized coordinates
X0, Y0 = sym.symbols(['X_{0}','Y_{0}']) # fixed position of first link

th1  ,  th2 = sym.symbols([       '\\theta_{1}',       '\\theta_{2}']) #positions
dth1 , dth2 = sym.symbols([ '\dot{\\theta}_{1}', '\dot{\\theta}_{2}']) #velocities
ddth1,ddth2 = sym.symbols(['\ddot{\\theta}_{1}','\ddot{\\theta}_{2}']) #accelerations

q   = sym.Matrix([  [th1],  [th2]]) #group into matrices
dq  = sym.Matrix([ [dth1], [dth2]])
ddq = sym.Matrix([[ddth1],[ddth2]])

#--------------------------------------------------------------------------------------------------

# STEP 1: system space coordinates written in terms of the generalised coordinates

# helper functions
def Rotate(v, th):
    # the 2D system space coordinates are [x;y;th], so we need a rotation function that can work with this
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [          0,            0, 1]]) # rotation matrix, augmented because of the theta element of the vector
    S = sym.Matrix([[0],[0],[th]]) # angle of rotation
    return R*v + S # coordinates after rotation

def GetXY(v):
    # this function is for applying a position-only offset (preserves the angle)
    vector_mask = sym.Matrix([[1],[1],[0]]) # gets rid of angle component of vector when multiplied elementwise
    return v.multiply_elementwise(vector_mask)

# positions of each link in their own reference frames
r0 = sym.Matrix([[X0],[Y0],[0]]) # position of the origin of the first link
r1_1 = sym.Matrix([[0],[-d1*l1],[0]]) # read as: position r1, in frame 1
r2_2 = sym.Matrix([[0],[-d2*l2],[0]]) # both point vertically down, with centres of mass halfway down the total length of the link

# positions of each link, moved into the inertial frame
r1_0 = Rotate(r1_1, th1) + GetXY(r0)

r2_1 = Rotate(r2_2, th2) + GetXY(r1_1/d1*(1-d1)) # GetXY of the bottom portion of link 1
r2_0 = Rotate(r2_1, th1) + GetXY(r1_0) # this code is improved upon in tut 3

## **The Jacobian**

The Jacobian takes the partial derivative of every element in one vector, with respect to every element in another vector, and produces a matrix containing all these partial derivatives. The Jacobian is an extremely important tool—a sort of 'Rosetta Stone' we use to translate vector quantities between coordinate representations. This means that we can use any vector quantity we know how to express in both sets of coordinates to find the Jacobian.

Let's consider an example:
{DEV}\[WIP: Image\]

<img src = "virtual_displacement.png" width = "250">

The position of the pendulum end in the world frame $x$ direction is:
$$x + \delta x = \mathrm{sin}(\theta + \delta \theta)$$

Using the <a href="https://en.wikipedia.org/wiki/List_of_trigonometric_identities#Angle_sum_and_difference_identities">compound angle identity</a>, this can be expanded to:
$$x + \delta x = \mathrm{sin}(\theta)\mathrm{cos}(\delta \theta) + \mathrm{cos}(\theta)\mathrm{sin}(\delta \theta)$$

Because the virtual displacement $\delta \theta$ is negligibly small, we can use the <a href="https://en.wikipedia.org/wiki/Small-angle_approximation">small angle theorem</a> to substitute $\mathrm{cos}(\delta \theta) = 1$ and $\mathrm{sin}(\delta \theta) = \delta \theta$, giving:
$$x + \delta x = \mathrm{sin}(\theta) + \mathrm{cos}(\theta)\delta \theta$$

Clearly, this can be split into the current position
$$x = \mathrm{sin}(\theta)$$
and the virtual displacement
$$\delta x = \mathrm{cos}(\theta)\delta \theta$$

This is a simple case where we are mapping a single direction to another, but this will typically be a mapping from vector to vector, of the form:
$$\mathbf{\delta r} = \mathbf{J} \mathbf{\delta q}$$
where $\mathbf{\delta r}$ is the generalized displacement vector expressed in the coordinates describing the external force, and $\mathbf{\delta q}$ is the generalized displacement of the generalized coordinates. The matrix $J$ that maps one coordinate frame to another is called the _Jacobian_.

In this example, the Jacobian has just one element—$\mathrm{cos}(\theta)$—but looking at the expression for $x$, we see that it is indeed the partial derivative $\frac{\delta x}{\delta \theta}$. The code block below (from last tutorial) shows the Jacobian being taken with vectors.

In [ ]:
# CODE FROM LAST TIME

# STEP 2: generate expressions for the system space velocities

dr1 = r1_0.jacobian(q)*dq
dr2 = r2_0.jacobian(q)*dq

# DISPLAY STUFF

# Link 1
from IPython.display import Latex

display(Latex(r"Position Vector (r) of link 1:"))
display(sym.trigsimp(r1_0))

display(Latex(r"Jacobian Matrix (J) of link 1:"))
display(sym.trigsimp(r1_0.jacobian(q)))

display(Latex(r"Derivative of Gen. Coords Vector (dq):"))
display(dq)

display(Latex(r"Velocity Vector (dr) of link 1:"))
display(sym.trigsimp(dr1))

# # Link 2 - Do uncomment to take a look
# display(Latex(r"Position Vector (r) of link 2:"))
# display(sym.trigsimp(r2_0))

# display(Latex(r"Jacobian Matrix (J) of link 2:"))
# display(sym.trigsimp(r2_0.jacobian(q)))

# display(Latex(r"Derivative of Gen. Coords Vector (dq):"))
# display(dq)

# display(Latex(r"Velocity Vector (dr) of link 2:"))
# display(sym.trigsimp(dr2))

In [ ]:
# CODE FROM LAST TIME

# STEP 3: generate expressions for the kinetic and potential energy

# helper functions
def Ek(m, In, dr):
    InM = sym.Matrix([[m,0,0],[0,m,0],[0,0,In]])
    return 0.5*dr.T*InM*dr

def Ep(m, r):
    return sym.Matrix([m*g*r[1]])

# expressions
T = Ek(m1, In1, dr1) + Ek(m2, In2, dr2)
V = Ep(m1, r1_0) + Ep(m2, r2_0)

#--------------------------------------------------------------------------------------------------

# STEP 4: calculate each term of the Lagrange equation

# term 1
Lg1 = sym.zeros(1,len(q))
for i in range(len(q)):
    dT_ddq = sym.diff(T,dq[i]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 2 = 0
# term 3
Lg3 = T.jacobian(q) # partial of T in q

# term 4
Lg4 = V.jacobian(q) # partial of U in q

## **Generalized Forces**

When no external forces act, the equations of motion are given by the formula:
$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}_{i}}\right)
-\frac{d}{dt}\left(\frac{\partial V}{\partial \dot{q}_{i}}\right)
-\frac{\partial T}{\partial q_i}
+\frac{\partial V}{\partial q_i}=0$$

Non-conservative forces are represented by the *generalized force* $Q_i$, which fits in on the other side of that equation:
$$\frac{d}{dt}\left(\frac{\partial T}{\partial \dot{q}_{i}}\right)
-\frac{d}{dt}\left(\frac{\partial V}{\partial \dot{q}_{i}}\right)
-\frac{\partial T}{\partial q_i}
+\frac{\partial V}{\partial q_i}=Q_{i}$$

### Generalized Force?

The generalized forces are a mapping of the external forces acting on the system into the generalized coordinates. We can think of them as a set of forces in the directions of the generalized coordinates that has an equivalent combined effect to the external forces.

Here, "equivalent" means "a set of forces that does the same work".

We know work is the product of a force and the displacement over which it moves the system: $W = F \Delta x$.

In an unconstrained system, the displacement of the body will simply coincide with the direction of the force, but this is not always the case for constrained systems. For example, consider a simplified version of the (unit length) pendulum system acted on by a horizontal force: 

<img src = "pendulum.png" width = "250">

The direction of the force is best expressed in terms of the world-frame $x$ axis, but the point at which the force acts is constrained to move in a circle around the fixed axel, so the direction it actually moves in is dependent on the current angle of the pendulum $\theta$. When the pendulum is at 0 degrees, the direction of motion instantaneously matches the direction of the force, but when it's at 90 degrees, it can't move horizontally at all, so no work can be done in that instant. 

To deal with this position dependence, it's helpful to introduce the idea of _virtual work_—the instantaneous work being done by the force. We express this in terms of a _virtual displacement_ as follows:
$$\delta W = F \delta x$$

The work and displacement are "virtual" because they can't physically exist: a force can only displace a body (and therefore, do work) by acting over time, so no actual displacement or work can happen in an infinitesimal instant. The virtual displacement is therefore more of an indication of the direction that the system is instantaneously being moved in, while the virtual work expresses how closely that direction aligns to the direction of the force.

In this pendulum example, an infinitesimal movement of the system in the positive direction of the force looks like this:

<img src = "virtual_displacement.png" width = "250">

What we're trying to do is find a new _generalized_ force $Q$, acting in the direction of our generalized coordinate, that produces equivalent virtual work, that is $\delta W = F \delta x = Q \delta q$.

Here, $\delta W = F \mathrm{cos}(\theta) \delta \theta$, so our generalized force must be $Q = F \mathrm{cos}(\theta)$.

Generalizing this to the vector case, we are actually getting the virtual work by calculating the dot product of a force vector $f$ and virtual displacement vector:
$\mathbf{\delta W} = \mathbf{f} \cdot \mathbf{\delta x}$.
Or, $\mathbf{\delta W} = \mathbf{f} \cdot \mathbf{J} \mathbf{\delta q}$.

This is the same as saying $\mathbf{\delta W} = \mathbf{J}^T \mathbf{f} \cdot \mathbf{\delta q}$, meaning: $$\mathbf{Q} = \mathbf{J}^T \mathbf{f}$$

Considering that the Jacobian mapped the generalized displacement from the generalized coordinate representation to the coordinates of the force vector, it makes a sort of intuitive sense that the transpose of the Jacobian maps the forces from their original coordinates to the generalized ones.

## **Example: Forces**

Now we can go back to the full-scale double pendulum system.

We'll describe the force in terms of the world frame coordinates:
$$\mathbf{f} = \begin{pmatrix} F_x\\
F_y\\
0\end{pmatrix}$$

To find the Jacobian, we need to find a vector where we know the relationship between the world frame and generalized coordinate expressions. The position of the end of the pendulum is a good candidate:{DEV}\[based on info already given above, may opt to remove "a good candidate" bc it might be more important to describe it as "the point at which the force acts".\]

$$\mathbf{r_{2}^{0}} = \begin{pmatrix} 
X_0 + l_1 sin(\theta_{1}) + l_2 sin(\theta_{1}+\theta_{2})\\ 
Y_0 - l_1 cos(\theta_{1}) - l_2 cos(\theta_{1}+\theta_{2})\\
\theta_{1} + \theta_{2} \end{pmatrix}$$

We then get the Jacobian by finding the partial derivatives:
$$J = \begin{pmatrix}
\frac{\partial r_{x}}{\partial \theta_1} & \frac{\partial r_{x}}{\partial \theta_2}\\
\frac{\partial r_{y}}{\partial \theta_1} & \frac{\partial r_{y}}{\partial \theta_2}\\
\frac{\partial r_{\theta}}{\partial \theta_1} & \frac{\partial r_{\theta}}{\partial \theta_2}
\end{pmatrix}
= \begin{pmatrix}
l_1 cos(\theta_1) + l_2 cos(\theta_1 + \theta_2) & l_2 cos(\theta_1 + \theta_2)\\
l_1 sin(\theta_1) + l_2 sin(\theta_1 + \theta_2) & l_2 sin(\theta_1 + \theta_2)\\
1 & 1\end{pmatrix}$$

The generalized force in both coordinates is given by $\mathbf{J}^T \mathbf{f}$
<!-- $$Q = \begin{pmatrix} F(l_1 cos(\theta_1) + l_2 cos(\theta_1 + \theta_2))\\
F(l_2 cos(\theta_1 + \theta_2)) \end{pmatrix}$$-->

$$Q = \begin{pmatrix} F_{x} \left(l_{1} cos(\theta_{1}) + l_{2} cos(\theta_{1} + \theta_{2})\right) + F_{y} \left(l_{1} sin(\theta_{1}) + l_{2} sin(\theta_{1} + \theta_{2})\right)\\
l_{2} \left(F_{x} cos(\theta_{1} + \theta_{2}) + F_{y} sin(\theta_{1} + \theta_{2})\right)\end{pmatrix}$$

## **Example: Torques**

The main takeaway that I hope you get from this section is that you can choose whichever coordinates are most convenient to represent the forces in the system. In Example 1, it made sense to use something other than the generalized coordinates, but often—especially in the case of the joint actuators—the forces can be expressed directly in these coordinates.

The torque acting on the second link of the pendulum is an example. It just acts in the direction $\theta_2$ so there is no need for a cross-coordinate mapping, but this is what one would looks like for the sake of completeness:

Force vector:
$$\mathbf{F} = \begin{pmatrix} 0 \\ \tau \end{pmatrix}$$

position vector:
$$\mathbf{r} = \begin{pmatrix} \theta_{1} \\ \theta_{2} \end{pmatrix}$$

Jacobian:
$$J = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$$

Generalized force:
$$\mathbf{Q} = \begin{pmatrix} 0 \\ \tau \end{pmatrix}$$

In [ ]:
# STEP 5: calculate generalized forces

# -----------------------------------------------------------------------------------

# arbitrary force
Fx, Fy = sym.symbols(['F_{x}','F_{y}'])
FF = sym.Matrix([[Fx],[Fy],[0]])

# end position in force coordinates wrt generalized coordinates:
rEnd_0 = Rotate(r2_2, th2 + th1) + GetXY(r2_0) #<- the first term is because r2_2 is at the midpoint of the link and we want the end

# the jacobian
JF = rEnd_0.jacobian(q)

# generalized force
QF = JF.T * FF # <- something.T is a shortcut for something.transpose()

# -----------------------------------------------------------------------------------

# arbitrary control torque
tau = sym.symbols('tau')
Qtau = sym.Matrix([[0],[tau]]) # <- no need for coordinate translation
# NOTE: in later tutorials we talk about absolute orientations, the above line is only correct for relative orientations

# -----------------------------------------------------------------------------------

# STEP 6: put it all together

Qtot = QF + Qtau
display(sym.trigsimp(Qtot))

EOM = Lg1 - Lg3 + Lg4 - Qtot.T
EOM = EOM.T
EOM = sym.trigsimp(EOM) # <- COMMENT OUT THIS LINE IF IT IS TOO SLOW

# display(EOM)

In [ ]:
N = 100
h = 0.01
t = np.arange(0,N*h,h)

#parameters
X0val, Y0val  = 0, 2
l1val, l2val  = 1, 1
d1val, d2val = 0.5, 0.5

# --------------------------------------------------------------------------------

# forces
Fy_val = 0;

# try both options!

# force but no torque - Option A
Fx_val = 5*np.sin(8*np.pi*t)
tau_val = np.zeros(np.shape(t));

# # torque but no force - Option B
# Fx_val = np.zeros(np.shape(t));
# tau_val = 5*np.sin(8*np.pi*t)

# --------------------------------------------------------------------------------

parameter_values = [(X0,X0val),(Y0,Y0val),
                    (g,9.81),
                    (m1,1),(m2,1),
                    (l1,l1val),(l2,l2val),
                    (d1,d1val),(d2,d2val),
                    (In1,0.08),(In2,0.08),
                    (Fy,Fy_val)] # since we aren't using Fy, subs here to make loop faster

# substitute parameters into EOM
EOM_sub_params = EOM.subs(parameter_values)

#initial conditions
th1vals  = [0]
th2vals  = [0]
dth1vals = [0]
dth2vals = [0]

for i in range(1,N+1):
    # conditions at previous timestep
    prev = [( th1,  th1vals[i-1]),
            ( th2,  th2vals[i-1]),
            (dth1, dth1vals[i-1]),
            (dth2, dth2vals[i-1])]
    
    # force and torque values
    forces = [(Fx,Fx_val[i-1]),(tau,tau_val[i-1])]
    
    # substitute previous conditions into EOM
    EOM_sub = EOM_sub_params.subs(prev).subs(forces)
    
    # solve for the acceleration
    accel = sym.solve(EOM_sub,[ddth1, ddth2])
    
    # integrate for the next velocity  
    dth1vals.append(float(dth1vals[i-1] + h * accel[ddth1]))
    dth2vals.append(float(dth2vals[i-1] + h * accel[ddth2]))
    
    # integrate for the next position
    th1vals.append(float(th1vals[i-1] + h * dth1vals[i]))
    th2vals.append(float(th2vals[i-1] + h * dth2vals[i]))
    
    print('\r%s' %i ,end='') # <- so you know how far into the simulation you are

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_pendulum(i,th1_in,th2_in,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-2,2])
    ax.set_ylim([0,4])
    
    #plot link 1
    L1topx = X0val
    L1topy = Y0val
    L1bottomx = X0val + l1val*np.sin(th1_in[i])
    L1bottomy = Y0val - l1val*np.cos(th1_in[i])
    ax.plot([L1topx,L1bottomx],[L1topy,L1bottomy],color='xkcd:blue')
    
    #plot link 2
    L2bottomx = L1bottomx + l2val*np.sin(th1_in[i] + th2_in[i])
    L2bottomy = L1bottomy - l2val*np.cos(th1_in[i] + th2_in[i]) 
    ax.plot([L1bottomx,L2bottomx],[L1bottomy,L2bottomy],color='xkcd:red')
    
update = lambda i: plot_pendulum(i,th1vals,th2vals,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(N),interval = 50,repeat=True) # interval = frame time. 1/50 = 20 fps
# animate = ani.FuncAnimation(fig1,update,range(N),interval = 1000*h,repeat=True) # if you want it to play at the actual speed

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook
# ani.Animation.save(animate,'pendulum_swing.mp4', fps=int(1/h), dpi=300) # if you want to save the animation instead of embedding it